# Лабораторная работа №3

ФИО:   
Группа: 

Отправлять можно следующими способами:
1. Запушить этот ноутбук в GitHub в репозиторий, где у вас лежат ноутбуки с лабами

Deadlines:
- Занятие №6 в семестре

Что необходимо сделать:  
- Обучить различного рода модели машинного обучения и сравнить их между собой  

---
## Читайте задание внимательно

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/1NOE0D4JQgD6LbvUqWboUI1TFj4P87ugbqUTDquxlGEI/edit?usp=sharing) необходимо узнать название своего датасета 
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1sbsjBsJ_ln0XgXCI9R6s17pvyvApgcwF?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (**если у вас многоклассовая классификация, прочтите P.S.S. внизу**).
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) (тут советую попробовать разные глубины деревьев)
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`) (тут тоже есть смысл попробовать разные `k`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`) 
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

---
P.S.  
Просьба -- делать каждое задание в отдельных ячейках и с отдельными заголовками (как пункт 1 и 2 в этом ноутбуке) типа  
- Заголовок
- Ячейки с кодом
- Другой заголовок
- Другие ячейки с кодом

P.S.S.  
Если вам повезло с многоклассовой классификацией, вам будет необходимо понять, умеет ли алгоритм работать с несколькими классами одновременно (обычно они не умеют). Поэтому вам может понадобиться такая штука, как OneVsRestClassifier ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier)), но советую ознакомиться с этой [страницей](https://scikit-learn.org/stable/modules/multiclass.html), здесь представлена более полная информация.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных
data = pd.read_csv('/Users/lekzyrs/Desktop/ailabs/data/spotify-2023.csv', sep=',', encoding='latin-1')

# Выводим информацию о загруженных данных
print(f"Размер датасета: {data.shape}")
print(f"\nПервые 5 строк:")
print(data.head())
print(f"\nИнформация о датасете:")
print(data.info())

Размер датасета: (952, 24)

Первые 5 строк:
                            track_name    artist(s)_name  artist_count  \
0  Seven (feat. Latto) (Explicit Ver.)  Latto, Jung Kook             2   
1                                 LALA       Myke Towers             1   
2                              vampire    Olivia Rodrigo             1   
3                         Cruel Summer      Taylor Swift             1   
4                       WHERE SHE GOES         Bad Bunny             1   

   released_year  released_month  released_day  in_spotify_playlists  \
0           2023               7            14                   553   
1           2023               3            23                  1474   
2           2023               6            30                  1397   
3           2019               8            23                  7858   
4           2023               5            18                  3133   

   in_spotify_charts    streams  in_apple_playlists  ...  bpm key   mode  \
0 

## 2. Понимаем, какая перед нами задача

In [2]:
# Определяем целевую переменную
target_variable = 'streams'

# Преобразуем streams в числовой формат (на случай, если есть строки с запятыми)
if target_variable in data.columns:
    data[target_variable] = data[target_variable].astype(str).str.replace(',', '').astype(float)

# Определяем тип задачи
unique_values = data[target_variable].nunique()
is_continuous = data[target_variable].dtype in ['float64', 'int64', 'float32', 'int32']

if is_continuous and unique_values > 20:
    task_type = "РЕГРЕССИЯ"
elif unique_values == 2:
    task_type = "БИНАРНАЯ КЛАССИФИКАЦИЯ"
else:
    task_type = "МНОГОКЛАССОВАЯ КЛАССИФИКАЦИЯ"

print(f"Тип задачи: {task_type}")


Тип задачи: РЕГРЕССИЯ


## 3. Делаем предобработку данных

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

# Определяем целевую переменную
target_variable = 'streams'

# Преобразуем streams в числовой формат (если еще не сделано)
if target_variable in data.columns:
    if data[target_variable].dtype == 'object':
        data[target_variable] = data[target_variable].astype(str).str.replace(',', '').astype(float)

# Разделяем на признаки и целевую переменную
X = data.drop(columns=[target_variable])
y = data[target_variable]

# Удаляем неинформативные колонки (названия треков и исполнителей)
X = X.drop(columns=['track_name', 'artist(s)_name'], errors='ignore')

# Преобразуем численные колонки, которые могут быть строками с запятыми
for col in X.columns:
    if X[col].dtype == 'object':
        # Пытаемся преобразовать в число (убираем запятые)
        try:
            X[col] = X[col].astype(str).str.replace(',', '').astype(float)
        except:
            pass  # Оставляем как есть, если не получается преобразовать

print(f"Размер признаков: {X.shape}")
print(f"Размер целевой переменной: {y.shape}")


Размер признаков: (952, 21)
Размер целевой переменной: (952,)


### 3.1. Разделение на train и test


In [4]:
# Разделение на train и test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер train выборки: {X_train.shape}")
print(f"Размер test выборки: {X_test.shape}")


Размер train выборки: (761, 21)
Размер test выборки: (191, 21)


### 3.2. Проверка и заполнение пропусков


In [5]:
# Проверяем пропуски в train выборке
print("Пропуски в train выборке:")
print(X_train.isnull().sum()[X_train.isnull().sum() > 0])

# Разделяем на численные и категориальные признаки
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"\nЧисленные признаки: {len(numeric_cols)}")
print(f"Категориальные признаки: {len(categorical_cols)}")

# Обучаем импьютеры на train выборке
# Для численных признаков - медиана
numeric_imputer = SimpleImputer(strategy='median')
X_train[numeric_cols] = numeric_imputer.fit_transform(X_train[numeric_cols])

# Для категориальных признаков - наиболее частое значение
categorical_imputer = SimpleImputer(strategy='most_frequent')
if categorical_cols:
    X_train[categorical_cols] = categorical_imputer.fit_transform(X_train[categorical_cols])

# Применяем импьютеры к test выборке
X_test[numeric_cols] = numeric_imputer.transform(X_test[numeric_cols])
if categorical_cols:
    X_test[categorical_cols] = categorical_imputer.transform(X_test[categorical_cols])

print("\nПропуски после заполнения:")
print(f"Train: {X_train.isnull().sum().sum()}")
print(f"Test: {X_test.isnull().sum().sum()}")


Пропуски в train выборке:
in_shazam_charts    42
key                 80
dtype: int64

Численные признаки: 19
Категориальные признаки: 2

Пропуски после заполнения:
Train: 0
Test: 0


### 3.3. Нормализация численных переменных


In [6]:
# Используем StandardScaler для нормализации численных признаков
scaler = StandardScaler()

# Обучаем на train и применяем к train и test
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

print("Численные признаки нормализованы с помощью StandardScaler")
print(f"Пример нормализованных данных (первые 3 строки, первые 3 признака):")
print(X_train[numeric_cols[:3]].head(3))


Численные признаки нормализованы с помощью StandardScaler
Пример нормализованных данных (первые 3 строки, первые 3 признака):
     artist_count  released_year  released_month
318     -0.611873       0.332593        1.113503
545     -0.611873      -0.027696       -0.866347
557     -0.611873       0.332593       -1.432018


### 3.4. Кодирование категориальных признаков


In [7]:
# Используем OneHotEncoder для категориальных признаков
if categorical_cols:
    ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    
    # Обучаем на train и применяем к train и test
    X_train_encoded = ohe.fit_transform(X_train[categorical_cols])
    X_test_encoded = ohe.transform(X_test[categorical_cols])
    
    # Создаем DataFrame с закодированными признаками
    feature_names = ohe.get_feature_names_out(categorical_cols)
    X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=feature_names, index=X_train.index)
    X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=feature_names, index=X_test.index)
    
    # Объединяем численные и закодированные категориальные признаки
    X_train_final = pd.concat([X_train[numeric_cols].reset_index(drop=True), 
                               X_train_encoded_df.reset_index(drop=True)], axis=1)
    X_test_final = pd.concat([X_test[numeric_cols].reset_index(drop=True), 
                              X_test_encoded_df.reset_index(drop=True)], axis=1)
    
    print(f"Категориальные признаки закодированы с помощью OneHotEncoder")
    print(f"Количество новых признаков после кодирования: {len(feature_names)}")
else:
    X_train_final = X_train[numeric_cols].reset_index(drop=True)
    X_test_final = X_test[numeric_cols].reset_index(drop=True)
    print("Категориальных признаков нет")

print(f"\nФинальный размер train: {X_train_final.shape}")
print(f"Финальный размер test: {X_test_final.shape}")
print(f"\nПервые 5 признаков:")
print(X_train_final.head())


Категориальные признаки закодированы с помощью OneHotEncoder
Количество новых признаков после кодирования: 11

Финальный размер train: (761, 30)
Финальный размер test: (191, 30)

Первые 5 признаков:
   artist_count  released_year  released_month  released_day  \
0     -0.611873       0.332593        1.113503      1.322189   
1     -0.611873      -0.027696       -0.866347     -0.528218   
2     -0.611873       0.332593       -1.432018     -0.528218   
3     -0.611873      -0.658202       -1.432018     -1.398997   
4      0.494150       0.332593        0.830667      0.886799   

   in_spotify_playlists  in_spotify_charts  in_apple_playlists  \
0             -0.593057          -0.517854           -0.510071   
1             -0.169046          -0.029881           -0.397661   
2             -0.407140          -0.615448           -0.307732   
3              1.174838           1.482833            0.220596   
4              0.407385           1.434036            1.659447   

   in_apple_charts 

## 4. Обучение моделей


### 4.1. Линейная модель (LinearRegression)


In [8]:
from sklearn.linear_model import LinearRegression

# Обучаем линейную модель
lr_model = LinearRegression()
lr_model.fit(X_train_final, y_train)

print("Линейная модель обучена")
print(f"Коэффициенты модели: {len(lr_model.coef_)} признаков")


Линейная модель обучена
Коэффициенты модели: 30 признаков


### 4.2. Деревянная модель (DecisionTreeRegressor)


In [9]:
from sklearn.tree import DecisionTreeRegressor

# Обучаем деревья с разными глубинами
tree_models = {}
depths = [3, 5, 10, 15, 20, None]

for depth in depths:
    tree_model = DecisionTreeRegressor(max_depth=depth, random_state=42)
    tree_model.fit(X_train_final, y_train)
    tree_models[f'depth_{depth}'] = tree_model
    print(f"Дерево с глубиной {depth} обучено")

print(f"\nВсего обучено {len(tree_models)} деревьев с разными глубинами")


Дерево с глубиной 3 обучено
Дерево с глубиной 5 обучено
Дерево с глубиной 10 обучено
Дерево с глубиной 15 обучено
Дерево с глубиной 20 обучено
Дерево с глубиной None обучено

Всего обучено 6 деревьев с разными глубинами


### 4.3. K-ближайших соседей (KNeighborsRegressor)


In [10]:
from sklearn.neighbors import KNeighborsRegressor

# Обучаем KNN с разными значениями k
knn_models = {}
k_values = [3, 5, 7, 10, 15, 20]

for k in k_values:
    knn_model = KNeighborsRegressor(n_neighbors=k)
    knn_model.fit(X_train_final, y_train)
    knn_models[f'k_{k}'] = knn_model
    print(f"KNN с k={k} обучен")

print(f"\nВсего обучено {len(knn_models)} моделей KNN с разными k")


KNN с k=3 обучен
KNN с k=5 обучен
KNN с k=7 обучен
KNN с k=10 обучен
KNN с k=15 обучен
KNN с k=20 обучен

Всего обучено 6 моделей KNN с разными k


### 4.4. Случайный лес (RandomForestRegressor)


In [11]:
from sklearn.ensemble import RandomForestRegressor

# Обучаем случайный лес
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_final, y_train)

print("Случайный лес обучен")
print(f"Количество деревьев: {rf_model.n_estimators}")


Случайный лес обучен
Количество деревьев: 100


## 5. Расчет метрик на train и test множествах


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

# Функции для расчета метрик регрессии
def calculate_metrics(y_true, y_pred):
    """Рассчитывает MAE, RMSE и MAPE"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    # MAPE - избегаем деления на ноль
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100
    return mae, rmse, mape

# Убеждаемся, что y_train и y_test имеют правильный формат
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print("Функции для расчета метрик готовы")


Функции для расчета метрик готовы


### 5.1. Метрики для LinearRegression


In [13]:
# Предсказания для LinearRegression
lr_train_pred = lr_model.predict(X_train_final)
lr_test_pred = lr_model.predict(X_test_final)

# Расчет метрик
lr_train_mae, lr_train_rmse, lr_train_mape = calculate_metrics(y_train, lr_train_pred)
lr_test_mae, lr_test_rmse, lr_test_mape = calculate_metrics(y_test, lr_test_pred)

print("Метрики для LinearRegression:")
print(f"Train - MAE: {lr_train_mae:.2f}, RMSE: {lr_train_rmse:.2f}, MAPE: {lr_train_mape:.2f}%")
print(f"Test  - MAE: {lr_test_mae:.2f}, RMSE: {lr_test_rmse:.2f}, MAPE: {lr_test_mape:.2f}%")


Метрики для LinearRegression:
Train - MAE: 188233390.40, RMSE: 283537751.09, MAPE: 15070.07%
Test  - MAE: 188047096.02, RMSE: 267344393.42, MAPE: 87.52%


### 5.2. Метрики для DecisionTreeRegressor


In [14]:
# Расчет метрик для всех деревьев
tree_results = []

for name, model in tree_models.items():
    train_pred = model.predict(X_train_final)
    test_pred = model.predict(X_test_final)
    
    train_mae, train_rmse, train_mape = calculate_metrics(y_train, train_pred)
    test_mae, test_rmse, test_mape = calculate_metrics(y_test, test_pred)
    
    tree_results.append({
        'Model': name,
        'Train_MAE': train_mae,
        'Train_RMSE': train_rmse,
        'Train_MAPE': train_mape,
        'Test_MAE': test_mae,
        'Test_RMSE': test_rmse,
        'Test_MAPE': test_mape
    })

tree_results_df = pd.DataFrame(tree_results)
print("Метрики для DecisionTreeRegressor:")
print(tree_results_df.to_string(index=False))


Метрики для DecisionTreeRegressor:
     Model    Train_MAE   Train_RMSE  Train_MAPE     Test_MAE    Test_RMSE  Test_MAPE
   depth_3 1.745528e+08 2.621711e+08 8378.056733 1.859178e+08 2.848736e+08  71.979945
   depth_5 1.296359e+08 1.888005e+08 8065.481355 1.658488e+08 2.552484e+08  60.237180
  depth_10 2.370750e+07 4.558421e+07   19.570319 1.670805e+08 2.488139e+08  51.237488
  depth_15 1.439779e+06 6.085118e+06    1.154526 1.661251e+08 2.496077e+08  51.906223
  depth_20 4.883778e+03 4.834459e+04    0.006434 1.753422e+08 2.594581e+08  53.376632
depth_None 0.000000e+00 0.000000e+00    0.000000 1.731944e+08 2.637060e+08  51.203564


### 5.3. Метрики для KNeighborsRegressor


In [15]:
# Расчет метрик для всех KNN моделей
knn_results = []

for name, model in knn_models.items():
    train_pred = model.predict(X_train_final)
    test_pred = model.predict(X_test_final)
    
    train_mae, train_rmse, train_mape = calculate_metrics(y_train, train_pred)
    test_mae, test_rmse, test_mape = calculate_metrics(y_test, test_pred)
    
    knn_results.append({
        'Model': name,
        'Train_MAE': train_mae,
        'Train_RMSE': train_rmse,
        'Train_MAPE': train_mape,
        'Test_MAE': test_mae,
        'Test_RMSE': test_rmse,
        'Test_MAPE': test_mape
    })

knn_results_df = pd.DataFrame(knn_results)
print("Метрики для KNeighborsRegressor:")
print(knn_results_df.to_string(index=False))


Метрики для KNeighborsRegressor:
Model    Train_MAE   Train_RMSE   Train_MAPE     Test_MAE    Test_RMSE  Test_MAPE
  k_3 1.652955e+08 2.455920e+08 10463.864175 2.242793e+08 3.212464e+08  88.894730
  k_5 1.865328e+08 2.785481e+08 13499.708228 2.204267e+08 3.133584e+08  86.813375
  k_7 1.956753e+08 2.934665e+08 15105.512020 2.114437e+08 3.029264e+08  87.533878
 k_10 2.053135e+08 3.095966e+08 11622.257911 2.126457e+08 3.025959e+08  91.581115
 k_15 2.144986e+08 3.239721e+08 15050.204381 2.156880e+08 3.086058e+08  97.086641
 k_20 2.215010e+08 3.368235e+08 17624.739003 2.247991e+08 3.195248e+08 100.051697


### 5.4. Метрики для RandomForestRegressor


In [16]:
# Предсказания для RandomForestRegressor
rf_train_pred = rf_model.predict(X_train_final)
rf_test_pred = rf_model.predict(X_test_final)

# Расчет метрик
rf_train_mae, rf_train_rmse, rf_train_mape = calculate_metrics(y_train, rf_train_pred)
rf_test_mae, rf_test_rmse, rf_test_mape = calculate_metrics(y_test, rf_test_pred)

print("Метрики для RandomForestRegressor:")
print(f"Train - MAE: {rf_train_mae:.2f}, RMSE: {rf_train_rmse:.2f}, MAPE: {rf_train_mape:.2f}%")
print(f"Test  - MAE: {rf_test_mae:.2f}, RMSE: {rf_test_rmse:.2f}, MAPE: {rf_test_mape:.2f}%")


Метрики для RandomForestRegressor:
Train - MAE: 53765728.56, RMSE: 88086213.82, MAPE: 6047.56%
Test  - MAE: 132754867.59, RMSE: 207724728.93, MAPE: 49.19%


### 5.5. Сводная таблица всех метрик


In [17]:
# Создаем сводную таблицу всех метрик
summary_results = []

# LinearRegression
summary_results.append({
    'Model': 'LinearRegression',
    'Train_MAE': lr_train_mae,
    'Train_RMSE': lr_train_rmse,
    'Train_MAPE': lr_train_mape,
    'Test_MAE': lr_test_mae,
    'Test_RMSE': lr_test_rmse,
    'Test_MAPE': lr_test_mape
})

# Лучшее дерево (по Test_RMSE)
best_tree = tree_results_df.loc[tree_results_df['Test_RMSE'].idxmin()]
summary_results.append({
    'Model': f"DecisionTree (best: {best_tree['Model']})",
    'Train_MAE': best_tree['Train_MAE'],
    'Train_RMSE': best_tree['Train_RMSE'],
    'Train_MAPE': best_tree['Train_MAPE'],
    'Test_MAE': best_tree['Test_MAE'],
    'Test_RMSE': best_tree['Test_RMSE'],
    'Test_MAPE': best_tree['Test_MAPE']
})

# Лучший KNN (по Test_RMSE)
best_knn = knn_results_df.loc[knn_results_df['Test_RMSE'].idxmin()]
summary_results.append({
    'Model': f"KNN (best: {best_knn['Model']})",
    'Train_MAE': best_knn['Train_MAE'],
    'Train_RMSE': best_knn['Train_RMSE'],
    'Train_MAPE': best_knn['Train_MAPE'],
    'Test_MAE': best_knn['Test_MAE'],
    'Test_RMSE': best_knn['Test_RMSE'],
    'Test_MAPE': best_knn['Test_MAPE']
})

# RandomForestRegressor
summary_results.append({
    'Model': 'RandomForestRegressor',
    'Train_MAE': rf_train_mae,
    'Train_RMSE': rf_train_rmse,
    'Train_MAPE': rf_train_mape,
    'Test_MAE': rf_test_mae,
    'Test_RMSE': rf_test_rmse,
    'Test_MAPE': rf_test_mape
})

summary_df = pd.DataFrame(summary_results)
print("=" * 100)
print("СВОДНАЯ ТАБЛИЦА МЕТРИК")
print("=" * 100)
print(summary_df.to_string(index=False))


СВОДНАЯ ТАБЛИЦА МЕТРИК
                        Model    Train_MAE   Train_RMSE   Train_MAPE     Test_MAE    Test_RMSE  Test_MAPE
             LinearRegression 1.882334e+08 2.835378e+08 15070.065914 1.880471e+08 2.673444e+08  87.519363
DecisionTree (best: depth_10) 2.370750e+07 4.558421e+07    19.570319 1.670805e+08 2.488139e+08  51.237488
             KNN (best: k_10) 2.053135e+08 3.095966e+08 11622.257911 2.126457e+08 3.025959e+08  91.581115
        RandomForestRegressor 5.376573e+07 8.808621e+07  6047.561545 1.327549e+08 2.077247e+08  49.193517


### 6. Сравнение метрик



1. **Какая модель справилась лучше:**

   Лучшей моделью является RandomForestRegressor, поскольку она имеет наименьшие значения метрик на тестовой выборке:

   Test MAE: 1.33e+08
   Test RMSE: 2.08e+08
   Test MAPE: 49.19%

2. **Имеет ли место переобучение?:**

   Да, переобучение имеет место у некоторых моделей:

   DecisionTreeRegressor: На обучении ошибка очень низкая (MAE = 2.37e+07), но на тесте она резко возрастает (MAE = 1.67e+08), что говорит о сильном переобучении.

   RandomForestRegressor также показывает заметную разницу между Train и Test, но менее выраженную, что указывает на умеренное переобучение.

3. **Имеет ли место недообучение?:**

   Да, недообучение имеет место у:

   LinearRegression: Ошибки на обучении и тесте практически одинаковые, но при этом очень высокие (MAPE > 87%), что говорит о недообучении.

   KNN: Аналогично, ошибки высокие на обеих выборках, что указывает на недостаточную обучаемость модели при текущей конфигурации

4. **Как можно улучшить метрики моделей?:**

   Feature Engineering: Добавить больше информативных признаков, попробовать полиномиальные признаки, логарифмирование целевой переменной и т.д.

   Подбор гиперпараметров:
   Для RandomForest: регулировать n_estimators, max_depth, min_samples_split.
   Для DecisionTree: ограничить глубину дерева, использовать min_samples_split, min_samples_leaf.

   Масштабирование признаков: Особенно важно для KNN.

   Кросс-валидация: Для уверенности в правильных параметрах и снижения риска переобучения.
